In [ ]:
# Importing libraries
import ee
import geemap
# Initialize the Earth Engine Library
ee.Initialize()

# Loading the area of interest with the basemap

# Load the shapefile asset
asset_id = 'projects/ee-ssouley/assets/Niamey'
region = ee.FeatureCollection(asset_id)
# Initialize the Map

Map = geemap.Map()
Map.add_basemap("HYBRID")
Map.centerObject(region)
Map.addLayer(region, {}, 'Niamey')
Map

# Set the date range

start_date = "2021-01-01"
end_date = "2024-12-31"

# Create a Sentinel-2 image composite

image = geemap.dynamic_world_s2(region, start_date, end_date)
vis_params = {"bands": ["B4", "B3", "B2"], "min": 0, "max": 3000}
Map.addLayer(image, vis_params, "Sentinel-2 image")

# Create Dynamic World Land Cover Composite and Clip it to the region

landcover = geemap.dynamic_world(region, start_date, end_date, return_type="hillshade")
clipped_landcover = landcover.clip(region)
Map.addLayer(clipped_landcover, {}, "Clipped Land Cover")

## Add legend to the map
#Map.add_legend(title= "Niamey Land Cover", builtin_legend = "Dynamic_World")

# Customize the legend

legend_dict = {
"Water": "#419BDF",
"Trees": "#397D49",
"Grass": "#88B053",
"Flooded Vegetation": "#7A87C6",
"Crops": "#E49635",
"Shurb & Scrub": "#DFC35A",
"Built": "#C4281B",
"Bare": "#A59B8F",
}

## custom legend
Map.add_legend(title = "Niamey Land Cover", legend_dict = legend_dict)
Map

# Export the clipped Land Cover To Google Drive 
export_image = clipped_landcover.visualize(**vis_params).clip(region)
task = ee.batch.Export.image.toDrive(
image = export_image,
description = "Clipped_Land_Cover",
fileNamePrefix = "EarthEngineExports",
region = region.geometry().bounds().getInfo()['coordinates'],
scale = 10,
maxPixels = 1e9
)

task.start()
import time
# Function to check task status 
def check_task_status(task):
    while task.active():
        status = task.status()
        print(f"Task status: {status['state']}")
        time.sleep(60)
#Print final task status
    print(f"Final task status: {task.status()['state']}")
#check the status of export task
check_task_status(task)
# Display the map
Map
